In [ ]:
from pathlib import Path
import geopandas as gpd
from matplotlib import pyplot as plt
import rioxarray

In [ ]:
f_tree_locations = Path('../../data/Tree_location1.gpkg')
data_image = Path('../../data/Fieldwork_data/PNM/gpkg_borore/area_recortada_mosaico.tif')

## Select labels

In [ ]:
tree_locs = gpd.read_file(f_tree_locations)

In [ ]:
def _is_round(row, num_vertex=10):
    # Polygons with >num_vertex vertex are considered to be round 
    return len(list(row['geometry'].exterior.coords))>num_vertex

In [ ]:
# Select squares from the tree labels
gdf_treelabels = tree_locs.where(~tree_locs.apply(_is_round, axis=1)).dropna(how='all')

In [ ]:
image_uav = rioxarray.open_rasterio(data_image).isel(band=range(3))

fig, ax = plt.subplots()
image_uav.plot.imshow()
gdf_treelabels.plot(ax=ax, facecolor="none", edgecolor="red")

## Make Crops

In [ ]:
from deepforest_exp import make_crops

# # Define crops by tif coordinates (a set of smaller tiles)
# crop_centers = [(331_050,7366_595), (331_175,7366_700), 
#                 (331_080,7366_780), (331_090,7366_815),
#                 (331_140,7366_820), (331_200,7366_845), 
#                 (331_210,7366_895), (331_280,7366_885),
#                 ]
# crop_sizes = 60

# Define crops by tif coordinates
crop_centers = [(331_050,7366_595), (331_175,7366_700), 
                (331_100,7366_790), (331_240,7366_870)
                ]
crop_sizes = 135

img_crops = make_crops(image_uav, crop_centers, crop_sizes)

In [ ]:
fig, axes = plt.subplots(2,2)
fig.set_size_inches(15, 15)
for img, ax in zip(img_crops, axes.flatten()):
    xmin, ymin, xmax, ymax = img.rio.bounds()
    boxes = gdf_treelabels.cx[xmin:xmax, ymin:ymax]
    img.plot.imshow(ax=ax)
    boxes.plot(ax=ax, facecolor="none", edgecolor="red")
    ax.set_aspect('equal', adjustable='box')
fig.tight_layout()

In [ ]:
# Predict with one set of parameters
from deepforest_exp import predict_crops

tile_size = 800
overlap_ratio = 0.35

gdf_prediction = predict_crops(img_crops, tile_size, overlap_ratio)
gdf_prediction.to_file(f'prediction/tile{tile_size}_overlap{overlap_ratio:.2f}.shp')

In [ ]:
fig, axes = plt.subplots(2,2)
fig.set_size_inches(10, 10)
for img, ax in zip(img_crops, axes.flatten()):
    xmin, ymin, xmax, ymax = img.rio.bounds()
    labels = gdf_treelabels.cx[xmin:xmax, ymin:ymax]
    predictions = gdf_prediction.cx[xmin:xmax, ymin:ymax]
    img.plot.imshow(ax=ax)
    labels.plot(ax=ax, facecolor="none", edgecolor="blue")
    predictions.plot(ax=ax, facecolor="none", edgecolor="red")
    ax.set_aspect('equal', adjustable='box')
fig.tight_layout()

## Select prediction results

In [ ]:
# Load the prediction
gdf_prediction =  gpd.read_file('prediction/crop_2200/tile800_overlap0.45.shp')

In [ ]:
# Check prediction score histogram
gdf_prediction['score'].hist(bins=40)

In [ ]:
gdf_prediction_subset = gdf_prediction[gdf_prediction['score']>0.3]
gdf_prediction_subset = gdf_prediction_subset[gdf_prediction_subset['geometry'].area>20]

In [ ]:
fig, axes = plt.subplots(2,2)
fig.set_size_inches(10, 10)
for img, ax in zip(img_crops, axes.flatten()):
    xmin, ymin, xmax, ymax = img.rio.bounds()
    labels = gdf_treelabels.cx[xmin:xmax, ymin:ymax]
    predictions = gdf_prediction_subset.cx[xmin:xmax, ymin:ymax]
    img.plot.imshow(ax=ax)
    labels.plot(ax=ax, facecolor="none", edgecolor="blue")
    predictions.plot(ax=ax, facecolor="none", edgecolor="red")
    ax.set_aspect('equal', adjustable='box')
fig.tight_layout()

## Other experiments

In [ ]:
# from deepforest_exp import predict_experiments

# tile_size = [1200, 1400]
# overlap_ratio = [0.25, 0.35]

# dict_prediction = predict_experiments(img_crops, tile_size, overlap_ratio)

In [ ]:
# outdir = Path('prediction')
# for key, item in dict_prediction.items():
#     item.to_file(outdir/ f'{key}.shp')

In [ ]:
# for file in outdir.glob('*.shp'):
#     gdf_prediction = gpd.read_file(file)

In [ ]:
# outdir = Path('prediction')

# for file in outdir.glob('*.shp'):
#     gdf_prediction = gpd.read_file(file)
#     fig, axes = plt.subplots(2,2)
#     fig.set_size_inches(15, 15)
#     for img, ax in zip(img_crops, axes.flatten()):
#         xmin, ymin, xmax, ymax = img.rio.bounds()
#         labels = gdf_treelabels.cx[xmin:xmax, ymin:ymax]
#         predictions = gdf_prediction.cx[xmin:xmax, ymin:ymax]
#         img.plot.imshow(ax=ax)
#         labels.plot(ax=ax, facecolor="none", edgecolor="blue")
#         predictions.plot(ax=ax, facecolor="none", edgecolor="red")
#         ax.set_aspect('equal', adjustable='box')
#     fig.tight_layout()
#     fig.savefig(outdir/ f'{file.stem}.png')